## This Notebook is for geometrics related searches and shows how the requests can be built

### Initialize

In [ ]:
import os
import sys
from datetime import datetime

import pandas as pd
import requests

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_objects as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

base_url = get_base_url()
root_path = get_root_path()
selected = sys.path[0].split(os.sep)[-2]



def execute_search_query(query):
    r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[query])
    if r.status_code == 200:
        res = r.json()[0]
        if '@odata.count' in res:
            print(f'Total count: {res["@odata.count"]}')
        if res and 'value' in res and res['value']:
            df = pd.DataFrame.from_dict(res['value'])[ ['entityDesc', 'position','street', 'houseNumber', 'city', 'postalCode', 'country']]
            df.rename(columns={'entityDesc': 'Description',
                  'position': 'Position', 'street': 'Street', 'houseNumber': 'House Number', 'city': 'City',
                           'postalCode': 'Postal Code', 'country': 'Country'}, inplace=True)
            display(df)
        else:
            print('No results')
    else:
        print(f'Error during search. HTTP status code {r.status_code}')
        print(r.json())


## Search for position 8.642383987644662,49.30643058918701

In [ ]:
so = esh.EshObject(
    count=True,
    top=2,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(property='position'),
                operator=esh.ComparisonOperator.Search,
                value=esh.Point(
                    coordinates=[8.642383987644662,49.30643058918701]))]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


## Search for the location coordinates of the person "Alex Kennedy"

In [ ]:
so = esh.EshObject(
    count=True,
    top=2,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'lastName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Kennedy')),
            esh.Comparison(
                property=esh.Property(
                    property=['relPerson', 'person', 'firstName']),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(
                    value='Alex'))
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


## Search for locations which is within 4 given coordinate points
--> A polygon(several coordinates) is defined and it is searched for locations which is within these coordinates

In [ ]:
so = esh.EshObject(
    count=True,
    top=2,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Location'),
            esh.Comparison(
                property=esh.Property(property='position'),
                operator=esh.ComparisonOperator.Search,
                value=esh.StringValue(value='COVERED_BY:POLYGON((-0.06162643432617188 51.51034504891232,-0.062055587768554694 51.512428331778096,-0.059190988540649414 51.512675381440225,-0.06162643432617188 51.51034504891232))'),
            )
        ]))

print(f'ESH query statement: {so.to_statement()}')
execute_search_query(so.dict(exclude_none=True))


## Search for the person who is within a defined polygon 


In [ ]:
so = esh.EshObject(
    count=True,
    top=1,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relLocation','location','position']),
                operator=esh.ComparisonOperator.Search,
                value= esh.StringValue(value='COVERED_BY:POLYGON((-0.06162643432617188 51.51034504891232,-0.062055587768554694 51.512428331778096,-0.059190988540649414 51.512675381440225,-0.06162643432617188 51.51034504891232))'),
                    )]))

print(f'ESH query statement: {so.to_statement()}')


r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
        df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())

## Search for all persons who are within london

In [49]:
so = esh.EshObject(
    count=True,
    top=6,
    searchQueryFilter=esh.Expression(
        operator=esh.LogicalOperator.AND,
        items=[
            esh.ScopeComparison(values='Person'),
            esh.Comparison(
                property=esh.Property(property=['relLocation','location','position']),
                operator=esh.ComparisonOperator.Search,
                value= esh.StringValue(value='COVERED_BY:POLYGON((-0.31173706054687506 51.618869218965926,0.10574340820312501 51.63762391020278,0.09887695312500001 51.36920841344186,-0.28976440429687506 51.40348936856666, -0.32135009765625006 51.5693878622646,-0.31173706054687506 51.618869218965926))'),
                    )]))

print(f'ESH query statement: {so.to_statement()}')


r = requests.post(f'{base_url}/v1/query/{selected}{TENANT_SUFFIX}/latest', json=[so.dict()])
if r.status_code == 200:
    res = r.json()[0]
    if '@odata.count' in res:
        print(f'Total count: {res["@odata.count"]}')
    if res and 'value' in res and res['value']:
        df = pd.DataFrame.from_dict(res['value'])[['firstName', 'lastName','age']]
        df.rename(columns={'firstName': 'First Name', 'lastName': 'Last Name', 'age':'Age'}, inplace=True)
        display(df)
    else:
        print('No results')
else:
    print(f'Error during search. HTTP status code {r.status_code}')
    print(r.json())

ESH query statement: /$all?$top=6&$count=true&$apply=filter(Search.search(query='SCOPE:Person AND relLocation.location.position:COVERED_BY:POLYGON((-0.31173706054687506 51.618869218965926,0.10574340820312501 51.63762391020278,0.09887695312500001 51.36920841344186,-0.28976440429687506 51.40348936856666, -0.32135009765625006 51.5693878622646,-0.31173706054687506 51.618869218965926))'))
Total count: 3


,First Name,Last Name,Age
0,Joseph,Kennedy,36
1,Alex,Kennedy,57
2,Jerry,Roberts,37
